### Convolutional Neural Network(CNN)



### Convolution Layer

- Convolution이 어떻게 동작하는지

- Maxpooling

- Convolution 계산

- 최종 처리(flatten and mlp)

### Convolution Layer

Convolution layer는 합성곱 연산을 이용하는 인공신경망입니다. MLP에서는 각각의 pixel에 대해서 weight를 지정해주지만, CNN에서는 하나의 공유하는 filter를 이용하여 대응하는 pixel들을 곱하는 방식으로 연산이 진행됩니다.

이미지를 MLP를 이용하여 연산을 하면, 이미지가 커질수록 그에 따른 weight수가 증가하기 때문에 연산량이 급격하게 늘어나지만, Convolution layer의 경우 공유되는 필터를 통해 더 효율적으로 처리를 할 수 있다는 장점이 있어 Computer vision 분야에서는 대부분의 모델을 Convolution layer를 이용합니다.

![image](https://cdn-images-1.medium.com/max/1600/1*EuSjHyyDRPAQUdKCKLTgIQ.png)

### Pooling Layer

Convolution layer를 통해 얻어진 값들은 pooling layer를 거쳐 downsampling이 됩니다. 평균값을 이용하는 Meanpooling 등 다양한 종류가 있지만 주로 최대값을 이용하는 Maxpooling을 이용하는데, 이를 통해 이미지의 중요한 feature들만 유지하여 연산량을 줄일 수 있게 됩니다.
Convolution을 거쳐 나온 activation maps이 있을때, 이를 이루는 convolution layer를 resizing해 새로운 layer를 얻는 것 ( 김성훈 교수님 강의 에서의 Pooling 정의)

![image](https://oopy.lazyrockets.com/api/v2/notion/image?src=https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F9b5327fe-d3a8-4f6c-8f43-0ea66973a27f%2FUntitled.png&blockId=b86a2d9d-d6f2-4ae6-920d-47d5438bf9fd)


### Padding

Padding이란 이미지의 각 모서리 부분에 특정 값을 채워 넣어주는 역할을 합니다.
Padding을 활용하는 이유는 아래와 같습니다.
- 이미지 축소를 막기 위해
- Edge pixel data를 충분히 활용하기 위해

일반적으로 Convolution layer를 진행하게 된다면 연산 결과가 기존의 이미지보다 작게 나오는 것을 확인하실 수 있습니다.

In [1]:
def cnn_param(W,F,P, S=1):
    """
    W = width or height
    F = Filters (Convolution filter의 크기)
    P = Padding 
    S = stride (몇칸씩 이동할 것인가)
    """
    return (((W - F + 2*P) / S) + 1 )

In [2]:
cnn_param(28, 3, 0, 1)

26.0

앞서 설명한것 처럼 Padding을 사용한다면 convolution을 통과한 결과값의 사이즈를 보존할 수 있을 뿐만 아니라, 모서리 부분의 feature까지 살릴 수 있다는 장점이 있습니다.

![image](https://miro.medium.com/max/325/1*b77nZmPH15dE8g49BLW20A.png)

In [3]:
cnn_param(28, 3, 1, 1)

28.0

### Fully Connected Layer

Convolution layer, pooling layer를 거치다 보면 중간 값들이 작아져 더이상 Convolution layer로의 분석이 의미가 없어지게 됩니다. 최종 의사결정을 마무리하기 위해 마지막 단의 결과를 벡터 형태로 펼쳐서 MLP를 보내게 됩니다. 이 과정이 일어나는 layer를 Fully connected layer라고 합니다.



이 모든 과정을 한번에 그리면 다음과 같이 됩니다.

![image](https://miro.medium.com/max/1400/1*u2FJVJpUtXN0IHSelbI94A.png)

### CNN on Tensorflow

In [1]:
import tensorflow as tf

In [2]:
## 약식
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

2022-07-28 17:14:09.254348: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
cnn.summary() # CNN summary를 통한 층 현황

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-traina

In [ ]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = tf.keras.layers.Conv2d(filter = 32, kernel_size = 3, activation = 'relu', padding = "same", input_shape = (28,28,1))
        self.pool1 = tf.keras.layers.MaxPooling2D((2,2))
        self.layer2 = tf.keras.layers.Conv2d(filter = 64, kernel_size = 3, activation = 'relu', padding = "same")
        self.pool2 = tf.keras.layers.MaxPooling2D((2,2))
        self.flat = tf.keras.layers.Flatten()
        self.fc = tf.keras.Dense(10, activation = "None")

    def call(self, x):
        out = self.layer1(x)
        out = self.pool1(out)
        out = self.layer2(out)
        out = self.pool2(out)
        out = self.flat(out)
        return self.fc(out)

### CNN on Pytorch

In [7]:
import torch
import torch.nn as nn

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = torch.nn.Linear(3 * 3 * 64, 10, bias=True)


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out